# COMP 551 Assignment 3 - Localization and Modified VGG Classifier

In [4]:
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mnist import MNIST

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.utils import plot_model, to_categorical
from keras.callbacks import History
from keras.optimizers import Nadam, Adam, SGD
from keras.utils import np_utils
from keras import backend as K
from keras import regularizers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# loading the original MNIST hand-written digits
mndata = MNIST('')
mndata.gz = True

images, labels = mndata.load_training()

In [2]:
imgs = np.asarray(images)
imgs = np.reshape(imgs, (-1, 28, 28))
# thresholds imgs
imgs = np.where(imgs < 100, 0, 255)

# plt.imshow(imgs[1], cmap='gray')
# plt.show()

In [5]:
def cnn():
    # create model
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(28,28,1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = cnn()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 30, 30, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)       

In [6]:
# X_train = np.array(imgs)/255
# X_train = X_train.reshape(-1, 28, 28, 1)

# onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
# y_train = onehot_encoder.fit_transform(np.reshape(labels, (-1, 1)))

# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.30, random_state=0, stratify=labels)
X_train = np.array(X_train)/255
X_test = np.array(X_test)/255

X_train = np.array(X_train).reshape(-1,28,28,1)
X_test = np.array(X_test).reshape(-1,28,28,1)

onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
y_train = onehot_encoder.fit_transform(np.reshape(y_train, (-1,1)))
y_test = onehot_encoder.transform(np.reshape(y_test, (-1,1)))

In [8]:
history = model.fit(X_train, y_train, batch_size=500, epochs=20)
model.save('localization_cnn.h5')

Epoch 1/20
42000/42000 [==============================] - ETA: 2:36 - loss: 0.3623 - accuracy: 0.89 - ETA: 2:28 - loss: 0.3698 - accuracy: 0.88 - ETA: 2:21 - loss: 0.3425 - accuracy: 0.89 - ETA: 2:15 - loss: 0.3306 - accuracy: 0.90 - ETA: 2:04 - loss: 0.3256 - accuracy: 0.90 - ETA: 1:54 - loss: 0.3263 - accuracy: 0.90 - ETA: 1:47 - loss: 0.3242 - accuracy: 0.90 - ETA: 1:51 - loss: 0.3132 - accuracy: 0.90 - ETA: 2:02 - loss: 0.3153 - accuracy: 0.90 - ETA: 2:10 - loss: 0.3260 - accuracy: 0.90 - ETA: 2:16 - loss: 0.3273 - accuracy: 0.90 - ETA: 2:20 - loss: 0.3280 - accuracy: 0.90 - ETA: 2:22 - loss: 0.3220 - accuracy: 0.90 - ETA: 2:21 - loss: 0.3139 - accuracy: 0.90 - ETA: 2:20 - loss: 0.3109 - accuracy: 0.90 - ETA: 2:17 - loss: 0.3108 - accuracy: 0.90 - ETA: 2:14 - loss: 0.3104 - accuracy: 0.90 - ETA: 2:11 - loss: 0.3063 - accuracy: 0.90 - ETA: 2:07 - loss: 0.3028 - accuracy: 0.90 - ETA: 2:04 - loss: 0.2999 - accuracy: 0.91 - ETA: 1:59 - loss: 0.2965 - accuracy: 0.91 - ETA: 2:00 - loss: 

42000/42000 [==============================] - ETA: 4:24 - loss: 0.0588 - accuracy: 0.98 - ETA: 4:21 - loss: 0.0631 - accuracy: 0.98 - ETA: 4:15 - loss: 0.0655 - accuracy: 0.98 - ETA: 4:03 - loss: 0.0644 - accuracy: 0.97 - ETA: 3:57 - loss: 0.0673 - accuracy: 0.97 - ETA: 3:49 - loss: 0.0678 - accuracy: 0.97 - ETA: 3:37 - loss: 0.0668 - accuracy: 0.97 - ETA: 3:26 - loss: 0.0688 - accuracy: 0.97 - ETA: 3:17 - loss: 0.0682 - accuracy: 0.97 - ETA: 3:06 - loss: 0.0661 - accuracy: 0.97 - ETA: 3:01 - loss: 0.0669 - accuracy: 0.97 - ETA: 2:55 - loss: 0.0653 - accuracy: 0.97 - ETA: 2:46 - loss: 0.0662 - accuracy: 0.97 - ETA: 2:42 - loss: 0.0703 - accuracy: 0.97 - ETA: 2:34 - loss: 0.0690 - accuracy: 0.97 - ETA: 2:30 - loss: 0.0700 - accuracy: 0.97 - ETA: 2:26 - loss: 0.0699 - accuracy: 0.97 - ETA: 2:21 - loss: 0.0697 - accuracy: 0.97 - ETA: 2:18 - loss: 0.0698 - accuracy: 0.97 - ETA: 2:12 - loss: 0.0698 - accuracy: 0.97 - ETA: 2:08 - loss: 0.0700 - accuracy: 0.97 - ETA: 2:04 - loss: 0.0708 - ac

42000/42000 [==============================] - ETA: 1:19 - loss: 0.0261 - accuracy: 0.98 - ETA: 1:32 - loss: 0.0294 - accuracy: 0.98 - ETA: 2:18 - loss: 0.0429 - accuracy: 0.98 - ETA: 2:37 - loss: 0.0408 - accuracy: 0.98 - ETA: 2:48 - loss: 0.0420 - accuracy: 0.98 - ETA: 2:54 - loss: 0.0452 - accuracy: 0.98 - ETA: 2:56 - loss: 0.0422 - accuracy: 0.98 - ETA: 2:53 - loss: 0.0396 - accuracy: 0.98 - ETA: 2:49 - loss: 0.0417 - accuracy: 0.98 - ETA: 2:44 - loss: 0.0434 - accuracy: 0.98 - ETA: 2:39 - loss: 0.0421 - accuracy: 0.98 - ETA: 2:34 - loss: 0.0417 - accuracy: 0.98 - ETA: 2:29 - loss: 0.0420 - accuracy: 0.98 - ETA: 2:24 - loss: 0.0426 - accuracy: 0.98 - ETA: 2:19 - loss: 0.0431 - accuracy: 0.98 - ETA: 2:12 - loss: 0.0430 - accuracy: 0.98 - ETA: 2:06 - loss: 0.0429 - accuracy: 0.98 - ETA: 2:01 - loss: 0.0450 - accuracy: 0.98 - ETA: 2:00 - loss: 0.0450 - accuracy: 0.98 - ETA: 2:02 - loss: 0.0448 - accuracy: 0.98 - ETA: 2:04 - loss: 0.0448 - accuracy: 0.98 - ETA: 2:05 - loss: 0.0441 - ac

42000/42000 [==============================] - ETA: 1:17 - loss: 0.0199 - accuracy: 0.99 - ETA: 2:04 - loss: 0.0214 - accuracy: 0.99 - ETA: 2:37 - loss: 0.0182 - accuracy: 0.99 - ETA: 2:54 - loss: 0.0246 - accuracy: 0.99 - ETA: 3:02 - loss: 0.0245 - accuracy: 0.99 - ETA: 3:05 - loss: 0.0266 - accuracy: 0.99 - ETA: 3:02 - loss: 0.0254 - accuracy: 0.99 - ETA: 2:58 - loss: 0.0277 - accuracy: 0.99 - ETA: 2:53 - loss: 0.0265 - accuracy: 0.99 - ETA: 2:47 - loss: 0.0258 - accuracy: 0.99 - ETA: 2:41 - loss: 0.0272 - accuracy: 0.99 - ETA: 2:36 - loss: 0.0273 - accuracy: 0.99 - ETA: 2:30 - loss: 0.0269 - accuracy: 0.99 - ETA: 2:25 - loss: 0.0265 - accuracy: 0.99 - ETA: 2:18 - loss: 0.0263 - accuracy: 0.99 - ETA: 2:13 - loss: 0.0275 - accuracy: 0.99 - ETA: 2:09 - loss: 0.0267 - accuracy: 0.99 - ETA: 2:04 - loss: 0.0281 - accuracy: 0.99 - ETA: 2:00 - loss: 0.0285 - accuracy: 0.99 - ETA: 1:57 - loss: 0.0281 - accuracy: 0.99 - ETA: 1:53 - loss: 0.0279 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0274 - ac

42000/42000 [==============================] - ETA: 3:59 - loss: 0.0310 - accuracy: 0.99 - ETA: 3:52 - loss: 0.0249 - accuracy: 0.99 - ETA: 3:44 - loss: 0.0245 - accuracy: 0.99 - ETA: 3:35 - loss: 0.0256 - accuracy: 0.99 - ETA: 3:23 - loss: 0.0255 - accuracy: 0.99 - ETA: 3:12 - loss: 0.0262 - accuracy: 0.99 - ETA: 3:03 - loss: 0.0239 - accuracy: 0.99 - ETA: 2:54 - loss: 0.0260 - accuracy: 0.99 - ETA: 2:46 - loss: 0.0252 - accuracy: 0.99 - ETA: 2:40 - loss: 0.0256 - accuracy: 0.99 - ETA: 2:33 - loss: 0.0255 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0241 - accuracy: 0.99 - ETA: 2:17 - loss: 0.0240 - accuracy: 0.99 - ETA: 2:10 - loss: 0.0231 - accuracy: 0.99 - ETA: 2:09 - loss: 0.0241 - accuracy: 0.99 - ETA: 2:11 - loss: 0.0233 - accuracy: 0.99 - ETA: 2:12 - loss: 0.0240 - accuracy: 0.99 - ETA: 2:14 - loss: 0.0245 - accuracy: 0.99 - ETA: 2:15 - loss: 0.0238 - accuracy: 0.99 - ETA: 2:14 - loss: 0.0238 - accuracy: 0.99 - ETA: 2:12 - loss: 0.0231 - accuracy: 0.99 - ETA: 2:10 - loss: 0.0226 - ac

In [10]:
score = model.evaluate(X_test, y_test, batch_size=1000)
print(score)

18000/18000 [==============================] - ETA:  - ETA:  - ETA: 10 - ETA: 14 - ETA: 15 - ETA: 16 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 27s 1ms/step
[0.02568431534907884, 0.9927777647972107]


In [11]:
print(score)

[0.02568431534907884, 0.9927777647972107]


In [12]:
train_labels = pd.read_csv('train_max_y.csv')
train_images = torch.load('win15Thresh210DigitData.pkl')
test_images = pd.read_pickle('test_max_x')

print(train_labels.head())
print(test_images.shape)

   Id  Label
0   0      6
1   1      7
2   2      2
3   3      9
4   4      7
(10000, 128, 128)


In [13]:
y_pred = []
for img in train_images:
    digit_pred = [];
    if(len(img)==0):
        y_pred.append(-1)
    else: 
        for digit in img:
            digit = np.where(digit > 0, 255, 0)
            digit = digit/255;
    #         print(np.argmax(model.predict(np.reshape(digit, (-1, 28, 28, 1)))))
    #         plt.imshow(digit, cmap='gray')
    #         plt.show()

            digit_pred.append(np.argmax(model.predict(np.reshape(digit, (-1, 28, 28, 1)))))
        y_pred.append(np.amax(digit_pred));

In [14]:
print(accuracy_score(train_labels['Label'], y_pred))

0.68258
